# Исследование базы данных сервиса вопросов и ответов StackOverflow

StackOverflow — сервис вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.

## Описание данных

Для анализа представлена версия базы, где хранятся данные о постах за 2008 год, но в таблицах есть также информация и о более поздних оценках, которые эти посты получили. 

**ER-диаграмма базы:**

![Схема БД](https://pictures.s3.yandex.net/resources/Frame_353_1_1664969443.png "Схема БД")

**Таблица stackoverflow.badges**


Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql. 

- id - Идентификатор значка, первичный ключ таблицы
- name - Название значка
- user_id - Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users
- creation_date - Дата присвоения значка


**Таблица stackoverflow.post_types**


Содержит информацию о типе постов. Их может быть два:

Question — пост с вопросом;
Answer — пост с ответом.


- id - Идентификатор поста, первичный ключ таблицы
- type - Тип поста

**Таблица stackoverflow.posts**

Содержит информацию о постах.


- id - Идентификатор поста, первичный ключ таблицы
- title - Заголовок поста
- creation_date - Дата создания поста
- favorites_count - Число, которое показывает, сколько раз пост добавили в «Закладки»
- last_activity_date - Дата последнего действия в посте, например комментария
- last_edit_date - Дата последнего изменения поста
- user_id - Идентификатор пользователя, который создал пост, внешний ключ к таблице users
- parent_id - Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом
- post_type_id - Идентификатор типа поста, внешний ключ к таблице post_types
- score - Количество очков, которое набрал пост
- views_count - Количество просмотров


**Таблица stackoverflow.users**

Содержит информацию о пользователях.

- id - Идентификатор пользователя, первичный ключ таблицы
- creation_date - Дата регистрации пользователя
- display_name - Имя пользователя
- last_access_date - Дата последнего входа
- location - Местоположение
- reputation - Очки репутации, которые получают за хорошие вопросы и полезные ответы
- views - Число просмотров профиля пользователя


**Таблица stackoverflow.vote_types**

Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько: 


UpMod — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
DownMod — такую отметку получают посты, которые показались пользователям наименее полезными.
Close — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
Offensive — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
Spam — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.


- id - Идентификатор типа голоса, первичный ключ
- name - Название метки

**Таблица stackoverflow.votes**


Содержит информацию о голосах за посты. 

- id - Идентификатор голоса, первичный ключ
- post_id - Идентификатор поста, внешний ключ к таблице posts
- user_id - Идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users
- bounty_amount - Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту
- vote_type_id - Идентификатор типа голоса, внешний ключ к таблице vote_types
- creation_date - Дата назначения голоса

## Подключение библиотек

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

## Подключение к БД

In [2]:
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-advanced-sql' # название базы данных
}  
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
) 
engine = create_engine(connection_string) 

## Запросы к БД

### Первая часть 

**1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».**

In [3]:
query = '''
SELECT COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id=1 AND (score>300 OR favorites_count>=100)
'''

pd.read_sql_query(query, con = engine) 


,count
0,1355


**Задание 2.
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.**

In [4]:
query = '''
SELECT ROUND(SUM(count_posts) / 18)
FROM
    (SELECT DATE_TRUNC('day', creation_date::date) AS dt, 
           COUNT(id) AS count_posts
    FROM stackoverflow.posts
    WHERE post_type_id=1 AND (creation_date BETWEEN '2008-11-01' AND '2008-11-19')
    GROUP BY DATE_TRUNC('day', creation_date::date)
    ORDER BY dt) cp;
'''

pd.read_sql_query(query, con = engine) 


,round
0,383.0


**Задание 3. 
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.**

In [5]:
query = '''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users u
     JOIN stackoverflow.badges b ON b.user_id=u.id
WHERE u.creation_date::date=b.creation_date::date;
'''

pd.read_sql_query(query, con = engine) 


,count
0,7047


**Задание 4.
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?**

In [6]:
query = '''
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts p
     JOIN stackoverflow.users u ON p.user_id=u.id
     JOIN stackoverflow.votes v ON p.id=v.post_id
WHERE u.display_name='Joel Coehoorn';
'''

pd.read_sql_query(query, con = engine) 

,count
0,12


**Задание 5.
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.**

In [7]:
query = '''
SELECT *,
       ROW_NUMBER() OVER (ORDER BY id DESC) rank
FROM stackoverflow.vote_types
ORDER BY id
'''

pd.read_sql_query(query, con = engine) 

,id,name,rank
0,1,AcceptedByOriginator,15
1,2,UpMod,14
2,3,DownMod,13
3,4,Offensive,12
4,5,Favorite,11
5,6,Close,10
6,7,Reopen,9
7,8,BountyStart,8
8,9,BountyClose,7
9,10,Deletion,6


**Задание 6.
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.**

In [8]:
query = '''
SELECT v.user_id,
       COUNT(v.id) cv
FROM stackoverflow.users u
     JOIN stackoverflow.votes v ON u.id=v.user_id
     JOIN stackoverflow.vote_types vt ON vt.id=v.vote_type_id
GROUP BY v.user_id, vt.name
HAVING vt.name='Close'
ORDER BY cv DESC, v.user_id DESC
LIMIT 10
'''

pd.read_sql_query(query, con = engine) 

,user_id,cv
0,20646,36
1,14728,36
2,27163,29
3,41158,24
4,24820,23
5,9345,23
6,3241,23
7,44330,20
8,38426,19
9,19074,19


**Задание 7.
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:**
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

**Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.**

In [9]:
query = '''
WITH ub AS
    (SELECT u.id uid,
            COUNT(DISTINCT b.id) b_count
     FROM stackoverflow.users u
     JOIN stackoverflow.badges b ON u.id=b.user_id
     WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
     GROUP BY 1)
SELECT uid,
       b_count,
       DENSE_RANK() OVER (ORDER BY b_count DESC)
FROM ub
ORDER BY 2 DESC, 1
LIMIT 10
'''

pd.read_sql_query(query, con = engine) 

,uid,b_count,dense_rank
0,22656,149,1
1,34509,45,2
2,1288,40,3
3,5190,31,4
4,13913,30,5
5,893,28,6
6,10661,28,6
7,33213,25,7
8,12950,23,8
9,25222,20,9


**Задание 8. 
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:**
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [10]:
query = '''
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE (title IS NOT NULL) AND (score != 0)
'''

pd.read_sql_query(query, con = engine) 

,title,user_id,score,round
0,Diagnosing Deadlocks in SQL Server 2005,1,82,573.0
1,How do I calculate someone's age in C#?,1,1743,573.0
2,Why doesn't IE7 copy <pre><code> blocks to the...,1,37,573.0
3,Calculate relative time in C#,1,1348,573.0
4,Wrapping StopWatch timing with a delegate or l...,1,92,573.0
...,...,...,...,...
47349,Multi-lingual projects in Visual Studio,3742716,1,1.0
47350,Change datatype when importing Excel file into...,3930756,5,5.0
47351,How to host a Silverlight app in a Sharepoint ...,4020932,8,8.0
47352,Getting counts for a paged SQL search stored p...,5696608,2,2.0


**Задание 9.
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.**

In [11]:
query = '''
WITH tb AS
    (SELECT u.id
    FROM stackoverflow.users u
         JOIN stackoverflow.badges b ON u.id=b.user_id
    GROUP BY u.id
    HAVING COUNT(DISTINCT b.id) > 1000)
SELECT p.title
FROM stackoverflow.posts p
     JOIN tb t ON t.id=p.user_id
WHERE p.title IS NOT NULL
'''

pd.read_sql_query(query, con = engine) 

,title
0,What's the hardest or most misunderstood aspec...
1,What's the strangest corner case you've seen i...
2,Project management to go with GitHub
3,What are the correct version numbers for C#?


**Задание 10. Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:**

- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

**Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.**

In [12]:
query = '''
SELECT id,
       views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           ELSE 3
       END AS group_views
FROM stackoverflow.users
WHERE (location LIKE '%%United States%%')
    AND views<>0
'''
pd.read_sql_query(query, con = engine) 

,id,views,group_views
0,3,24396,1
1,13,35414,1
2,23,757,1
3,25,3837,1
4,36,505,1
...,...,...,...
2359,354142,6,3
2360,335806,4,3
2361,129753,105,2
2362,182061,52,3


**Задание 11. 
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.**

In [13]:
query = '''
WITH users AS (
    SELECT id,
           views,
           CASE
               WHEN views >= 350 THEN 1
               WHEN views < 350 AND views >= 100 THEN 2
               ELSE 3
           END AS group_views
    FROM stackoverflow.users
    WHERE (location LIKE '%%United States%%')
        AND views<>0) 

SELECT id,
       group_views,
       views
FROM (
    SELECT *,
           max(views) OVER (PARTITION BY group_views) AS max_views
    FROM users) u
WHERE views = max_views
ORDER BY views DESC,
         id
'''

pd.read_sql_query(query, con = engine) 

,id,group_views,views
0,16587,1,62813
1,9094,2,349
2,9585,2,349
3,15079,2,349
4,33437,2,349
5,3469,3,99
6,4829,3,99
7,19006,3,99
8,22732,3,99
9,403434,3,99


**Задание 12. 
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:**
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [14]:
query = '''
WITH tab AS    
    (SELECT EXTRACT(DAY FROM creation_date) dt,
           COUNT(id) sum_user_dt
    FROM stackoverflow.users 
    WHERE EXTRACT(YEAR FROM creation_date) = 2008 AND EXTRACT(MONTH FROM creation_date) = 11
    GROUP BY dt)
SELECT dt,
       sum_user_dt,
       SUM(sum_user_dt) OVER (ORDER BY dt) AS sum_users
FROM tab;
'''

pd.read_sql_query(query, con = engine) 

,dt,sum_user_dt,sum_users
0,1.0,34,34.0
1,2.0,48,82.0
2,3.0,75,157.0
3,4.0,192,349.0
4,5.0,122,471.0
5,6.0,132,603.0
6,7.0,104,707.0
7,8.0,42,749.0
8,9.0,45,794.0
9,10.0,93,887.0


**Задание 13. 
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:**
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [15]:
query = '''
SELECT id,
       p_cd - u_cd AS days
FROM
    (SELECT DISTINCT p.user_id id,
                    FIRST_VALUE(p.creation_date) OVER (PARTITION BY p.user_id ORDER BY p.creation_date) p_cd,
                    FIRST_VALUE(u.creation_date) OVER (PARTITION BY u.id ORDER BY u.creation_date) u_cd
    FROM stackoverflow.posts p
    JOIN stackoverflow.users u ON p.user_id=u.id) tab;
'''

pd.read_sql_query(query, con = engine) 

,id,days
0,1,0 days 09:18:29
1,2,0 days 14:37:03
2,3,3 days 16:17:09
3,4,15 days 05:44:22
4,5,1 days 14:57:51
...,...,...
18844,3644960,0 days 00:00:00
18845,3742716,66 days 21:15:48
18846,3930756,8 days 02:11:55
18847,4020932,47 days 18:30:34


### Вторая часть

**Задание 1. 
Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.**

In [16]:
query = '''
SELECT DATE_TRUNC('month', creation_date)::date mn,
       SUM(views_count) sm
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY mn
ORDER BY sm DESC;
'''

pd.read_sql_query(query, con = engine) 

,mn,sm
0,2008-09-01,452928568
1,2008-10-01,365400138
2,2008-11-01,221759651
3,2008-12-01,197792841
4,2008-08-01,131367083
5,2008-07-01,669895


**Задание 2.
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.**

In [17]:
query = '''
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
       FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id=p.user_id
JOIN stackoverflow.post_types pt ON pt.id=p.post_type_id
WHERE pt.type='Answer' AND (DATE_TRUNC('day', p.creation_date)::date <= DATE_TRUNC('day', u.creation_date)::date + INTERVAL '1 month')
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY u.display_name;
'''

pd.read_sql_query(query, con = engine) 

,display_name,count
0,1800 INFORMATION,1
1,Adam Bellaire,1
2,Adam Davis,1
3,Adam Liss,1
4,Alan,8
...,...,...
74,lomaxx,1
75,mattlant,1
76,paxdiablo,1
77,tvanfosson,1


**Задание 3. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.**

In [18]:
query = '''
WITH p AS
(SELECT id,
        user_id,
        DATE_TRUNC('month', creation_date)::date p_dt
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date) = 2008),

us AS
(SELECT DISTINCT u.id
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id=p.user_id
WHERE DATE_TRUNC('month', u.creation_date)::date BETWEEN '2008-09-01' AND '2008-09-30'
AND DATE_TRUNC('month', p.creation_date)::date BETWEEN '2008-12-01' AND '2008-12-31')

SELECT p.p_dt,
       COUNT(p.id)
FROM p
JOIN us ON p.user_id=us.id
GROUP BY p.p_dt
ORDER BY p.p_dt DESC;
'''

pd.read_sql_query(query, con = engine) 

,p_dt,count
0,2008-12-01,17641
1,2008-11-01,18294
2,2008-10-01,27171
3,2008-09-01,24870
4,2008-08-01,32


**Задание 4.
Используя данные о постах, выведите несколько полей:**
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [19]:
query = '''
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id;
'''

pd.read_sql_query(query, con = engine) 

,user_id,creation_date,views_count,sum
0,1,2008-07-31 23:41:00,480476,480476
1,1,2008-07-31 23:55:38,136033,616509
2,1,2008-07-31 23:56:41,0,616509
3,1,2008-08-04 02:45:08,0,616509
4,1,2008-08-04 04:31:03,0,616509
...,...,...,...,...
243791,5696608,2008-12-23 16:00:37,0,2804
243792,5696608,2008-12-23 17:35:09,0,2804
243793,5696608,2008-12-24 01:02:48,0,2804
243794,5696608,2008-12-30 14:34:45,0,2804


**Задание 5. 
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.**

In [20]:
query = '''
SELECT ROUND(SUM(cd) / COUNT(user_id))
FROM
    (SELECT user_id,
           COUNT(DISTINCT DATE_TRUNC('day', creation_date)::date) cd --количество дней с 1 по 7 декабря 
    FROM stackoverflow.posts
    WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-12-01' AND '2008-12-07'
    GROUP BY user_id) us_days;
'''

pd.read_sql_query(query, con = engine) 

,round
0,2.0


**Задание 6. 
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:**
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [21]:
query = '''
SELECT dt,
       count_p,
       ROUND((count_p::numeric / LAG(count_p, 1) OVER (ORDER BY dt)- 1) * 100, 2) AS post_growth
FROM
   (SELECT EXTRACT(MONTH FROM creation_date) dt,
           COUNT(id) count_p
    FROM stackoverflow.posts
    WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-09-01' AND '2008-12-31'
    GROUP BY dt) tab
'''

pd.read_sql_query(query, con = engine) 

,dt,count_p,post_growth
0,9.0,70371,NaN
1,10.0,63102,-10.33
2,11.0,46975,-25.56
3,12.0,44592,-5.07


**Задание 7.
Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. 
Выведите данные за октябрь 2008 года в таком виде:**
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [22]:
query = '''
WITH a_user AS
    (SELECT user_id AS active_user
     FROM
        (SELECT user_id,
               COUNT(id) cp
        FROM stackoverflow.posts 
        GROUP BY user_id
        ORDER BY cp DESC) tab
    LIMIT 1)   
SELECT DISTINCT DATE_PART('week', ps.creation_date),
       MAX(ps.creation_date) OVER (ORDER BY DATE_TRUNC('week', ps.creation_date)::date)
FROM stackoverflow.posts ps
JOIN a_user ON ps.user_id = a_user.active_user
WHERE EXTRACT(MONTH FROM ps.creation_date) = 10; 
'''

pd.read_sql_query(query, con = engine) 

,date_part,max
0,40.0,2008-10-05 09:00:58
1,41.0,2008-10-12 21:22:23
2,42.0,2008-10-19 06:49:30
3,43.0,2008-10-26 21:44:36
4,44.0,2008-10-31 22:16:01
